In [1]:
!pip install transformers datasets

In [2]:
from datasets import load_dataset

import torchmetrics

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import pandas as pd
import torch
import os

from tools import download_dataset, load_dataset_into_to_dataframe, partition_dataset, IMDBDataset

In [3]:


## Load dataset
download_dataset()

data_frame = load_dataset_into_to_dataframe()
partition_dataset(data_frame)

df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test.csv")
## Tokenization
imdb_ds = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "val.csv",
        "test": "test.csv",
    },
)


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print(f"Tokenizer input max length: {tokenizer.model_max_length}")
print(f"Tokenizer vocabulary size: {tokenizer.vocab_size}")


def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)


imdb_tokenized = imdb_ds.map(tokenize_text, batched=True, batch_size=None)
del imdb_ds
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])


os.environ["TOKENIZERS_PARALLELISM"] = "false"
## Setup Dataloaders
train_ds = IMDBDataset(imdb_tokenized, partition_key="train")
val_ds = IMDBDataset(imdb_tokenized, partition_key="validation")
test_ds = IMDBDataset(imdb_tokenized, partition_key="test")
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=12,
    shuffle=True,
    num_workers=4,
)

val_loader = DataLoader(
    dataset=val_ds,
    batch_size=12,
    num_workers=4,
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=12,
    num_workers=4,
)


class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate=3e-5):
        super().__init__()
        self._learning_rate = learning_rate
        self._model = model

        self._val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self._test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

    def forward(self, input_ids, attention_mask, labels):
        return self._model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        self.log("val_loss", outputs["loss"], prog_bar=True)
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self._val_acc(predicted_labels, batch["label"])
        self.log("val_acc", self._val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self._test_acc(predicted_labels, batch["label"])
        self.log("accuracy", self._test_acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self._learning_rate)
        return optimizer



callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )
]
logger = CSVLogger(save_dir="logs/", name="lt_model")

## Finetune all layers (better do it on gpu)
all_layers_finetune_model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=2)
lt_model2 = LightningModel(all_layers_finetune_model)

trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="auto",
    precision="16-mixed",
    devices="auto",
    logger=logger,
    log_every_n_steps=10,
)

trainer.fit(model=lt_model2,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader,
            )
trainer.test(lt_model2, dataloaders=train_loader, ckpt_path="best")
trainer.test(lt_model2, dataloaders=val_loader, ckpt_path="best")
trainer.test(lt_model2, dataloaders=test_loader, ckpt_path="best")

100%|██████████| 50000/50000 [05:19<00:00, 156.36it/s]


Class distribution:


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │     0.508400022983551     │
└───────────────────────────┴───────────────────────────┘

Restoring states from the checkpoint path at logs/lt_model\version_3\checkpoints\epoch=1-step=5834.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/lt_model\version_3\checkpoints\epoch=1-step=5834.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.5005999803543091     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.5005999803543091}]